In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Load the data
store = pd.read_csv('store.csv')
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv', low_memory=False)

In [2]:
# Merge the DataFrames on the 'Store' column
trainStore = pd.merge(train, store, on='Store')
trainStore.to_csv('trainStore.csv', index=False)

In [3]:
trainStore = trainStore.dropna()
trainStore.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
942,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
943,2,4,2015-07-30,5567,601,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
944,2,3,2015-07-29,6402,727,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
945,2,2,2015-07-28,5671,646,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
946,2,1,2015-07-27,6627,638,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"


In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
trainStore_encoded_label = trainStore.copy()

for column in trainStore_encoded_label.columns:
    if trainStore_encoded_label[column].dtype == 'object':
        trainStore_encoded_label[column] = label_encoder.fit_transform(trainStore_encoded_label[column])

trainStore_encoded_label.head()


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
942,2,5,941,6064,625,1,1,0,1,0,0,570.0,11.0,2007.0,1,13.0,2010.0,1
943,2,4,940,5567,601,1,1,0,1,0,0,570.0,11.0,2007.0,1,13.0,2010.0,1
944,2,3,939,6402,727,1,1,0,1,0,0,570.0,11.0,2007.0,1,13.0,2010.0,1
945,2,2,938,5671,646,1,1,0,1,0,0,570.0,11.0,2007.0,1,13.0,2010.0,1
946,2,1,937,6627,638,1,1,0,1,0,0,570.0,11.0,2007.0,1,13.0,2010.0,1


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Convert pandas DataFrame to PyTorch tensors
X = torch.tensor(trainStore_encoded_label.drop(columns=['Sales']).values, dtype=torch.float32).to(device)
y = torch.tensor(trainStore_encoded_label['Sales'].values, dtype=torch.float32).to(device)

# Define a simple feedforward neural network
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Define hyperparameters
input_size = X.shape[1]
hidden_size = 64
output_size = 1
learning_rate = 0.001
num_epochs = 10
batch_size = 64

# Create DataLoader for batch training
dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize model, loss function, and optimizer
model = NeuralNetwork(input_size, hidden_size, output_size).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False):
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1))
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

print("Training finished.")

# Save the trained model
torch.save(model.state_dict(), 'sales_prediction_model.pth')

# Compute Mean Squared Error (MSE)
with torch.no_grad():
    model.eval()
    predictions = model(X)
    mse = nn.MSELoss()(predictions, y.unsqueeze(1))
    print(f"Mean Squared Error (MSE): {mse.item():.4f}")


Epoch [1/10], Loss: 2374847.2352


Epoch [2/10], Loss: 1342378.9928


Epoch [3/10], Loss: 1246846.5906


Epoch [4/10], Loss: 1169373.4389


Epoch [5/10], Loss: 1118967.3834


Epoch [6/10], Loss: 1078572.1077


Epoch [7/10], Loss: 1048237.0514


Epoch [8/10], Loss: 1024187.3770


Epoch [9/10], Loss: 1005037.6761


Epoch [10/10], Loss: 991389.5772
Training finished.
Mean Squared Error (MSE): 967340.0625
